In [11]:
# Dependencies and Setup
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress
from config import api_key
from config import g_key

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [3]:
# Load CSV file generated from WeatherPy Folder
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head(20)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,hvide%20sande,87,DK,1579932187,100,56.00,8.13,43.00,6.93
1,1,camana,100,PE,1579932095,86,-16.62,-72.71,66.47,2.21
2,2,mataura,71,NZ,1579932187,60,-46.19,168.86,66.00,10.00
3,3,brae,100,GB,1579932187,87,60.40,-1.35,46.40,26.40
4,4,chokurdakh,18,RU,1579932187,95,70.63,147.92,-35.27,1.97
5,5,grand%20gaube,75,MU,1579932187,100,-20.01,57.66,84.99,14.99
6,6,quatre%20cocos,75,MU,1579932188,100,-20.21,57.76,84.99,14.99
7,7,yellowknife,90,CA,1579932188,84,62.46,-114.35,0.00,9.17
8,8,mount%20gambier,75,AU,1579932188,53,-37.83,140.77,75.20,17.22
9,9,afsin,0,TR,1579932188,77,38.25,36.91,8.51,4.12


In [4]:
#cut unneeded columns
to_drop = ['Unnamed: 0']
weather_data.drop(to_drop, inplace=True, axis=1)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hvide%20sande,87,DK,1579932187,100,56.00,8.13,43.00,6.93
1,camana,100,PE,1579932095,86,-16.62,-72.71,66.47,2.21
2,mataura,71,NZ,1579932187,60,-46.19,168.86,66.00,10.00
3,brae,100,GB,1579932187,87,60.40,-1.35,46.40,26.40
4,chokurdakh,18,RU,1579932187,95,70.63,147.92,-35.27,1.97
...,...,...,...,...,...,...,...,...,...
557,bedele,84,ET,1579932245,34,8.46,36.35,71.78,1.25
558,port%20moresby,40,PG,1579932245,63,-9.44,147.18,91.40,11.41
559,antofagasta,75,CL,1579932041,94,-23.65,-70.40,71.60,3.36
560,bata,20,GQ,1579932245,100,1.86,9.77,77.00,2.91


Humidity Heatmap

Configure gmaps. Use the Lat and Lng as locations and Humidity as the weight. Add Heatmap layer to map.

In [6]:
locations = weather_data[["Lat", "Lng"]].astype(float)
# locations

humidity = weather_data["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions. Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
116,sakhipur,71.60,3.36,0
150,salalah,78.80,2.24,0
209,tura,72.73,4.12,0
446,upington,71.60,5.82,0
475,chibuto,78.82,5.39,0
489,badarpur,70.83,4.88,0


Hotel Map

Store into variable named hotel_df. Add a "Hotel Name" column to the DataFrame. Set parameters to search for hotels with 5000 meters. Hit the Google Places API for each city's coordinates. Store the first Hotel result into the DataFrame. Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,hvide%20sande,DK,56.00,8.13,
1,camana,PE,-16.62,-72.71,
2,mataura,NZ,-46.19,168.86,
3,brae,GB,60.40,-1.35,
4,chokurdakh,RU,70.63,147.92,
...,...,...,...,...,...
557,bedele,ET,8.46,36.35,
558,port%20moresby,PG,-9.44,147.18,
559,antofagasta,CL,-23.65,-70.40,
560,bata,GQ,1.86,9.77,


In [9]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,hvide%20sande,DK,56.00,8.13,Hvide Sande
1,camana,PE,-16.62,-72.71,Cercado de Camana
2,mataura,NZ,-46.19,168.86,Mataura
3,brae,GB,60.40,-1.35,Brae
4,chokurdakh,RU,70.63,147.92,Chokurdakh


In [9]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))